In [26]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
application = pd.read_csv('data/application_train.csv')
application.sample(7, random_state=57)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
170430,297531,0,Cash loans,M,Y,N,0,157500.0,679500.0,27076.5,679500.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.026392,-14705,-7272,-6238.0,-220,8.0,1,1,0,1,1,0,Drivers,2.0,2,2,SATURDAY,17,0,0,0,0,0,0,Industry: type 7,0.342672,0.570747,0.332851,0.1206,NaN,0.9916,NaN,NaN,0.16,0.1379,0.3333,NaN,NaN,NaN,0.1478,NaN,0.0037,0.1229,NaN,0.9916,NaN,NaN,0.1611,0.1379,0.3333,NaN,NaN,NaN,0.1540,NaN,0.0039,0.1218,NaN,0.9916,NaN,NaN,0.16,0.1379,0.3333,NaN,NaN,NaN,0.1504,NaN,0.0037,NaN,block of flats,0.1170,Others,No,0.0,0.0,0.0,0.0,-2252.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
77136,189440,0,Cash loans,F,Y,Y,0,220500.0,432567.0,19183.5,328500.0,Family,Working,Secondary / secondary special,Separated,House / apartment,0.018029,-20590,-708,-1768.0,-4081,11.0,1,1,0,1,0,0,Laborers,1.0,3,3,TUESDAY,13,0,0,0,0,0,0,Government,0.788965,0.090318,0.764939,0.0722,NaN,0.9846,NaN,NaN,0.00,0.0345,0.2083,NaN,NaN,NaN,0.0422,NaN,0.0000,0.0735,NaN,0.9846,NaN,NaN,0.0000,0.0345,0.2083,NaN,NaN,NaN,0.0439,NaN,0.0000,0.0729,NaN,0.9846,NaN,NaN,0.00,0.0345,0.2083,NaN,NaN,NaN,0.0429,NaN,0.0000,NaN,block of flats,0.0395,Panel,No,0.0,0.0,0.0,0.0,-119.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,7.0
236101,373468,0,Cash loans,F,Y,Y,0,90000.0,254700.0,14350.5,225000.0,Family,Pensioner,Secondary / secondary special,Married,House / apartment,0.035792,-23878,365243,-9575.0,-4358,15.0,1,0,0,1,1,0,NaN,2.0,2,2,MONDAY,13,0,0,0,1,0,0,XNA,0.841625,0.295620,0.656158,0.0897,NaN,0.9816,NaN,NaN,NaN,0.2069,0.1667,NaN,0.0908,NaN,0.0766,NaN,0.0098,0.0914,NaN,0.9816,NaN,NaN,NaN,0.2069,0.1667,NaN,0.0928,NaN,0.0799,NaN,0.0104,0.0906,NaN,0.9816,NaN,NaN,NaN,0.2069,0.1667,NaN,0.0923,NaN,0.0780,NaN,0.0100,NaN,block of flats,0.0700,"Stone, bric

In [ ]:
def process_application(application_df):
    df = application_df.copy()
    df.loc[df['CODE_GENDER'] == 'XNA', 'CODE_GENDER'] = 'F'
    

In [4]:
FLAG_DOCUMENT_GROUP = ['FLAG_DOCUMENT_%s' % i for i in range(2, 22)]
TARGET_GROUP = ['TARGET']
GENERAL_BIO_GROUP = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
                     'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
                     'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
                     'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE',
                     'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'ORGANIZATION_TYPE',
                     'OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']
DAYS_BIO_GROUP = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']
AMT_GROUP = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE'] + [
    'AMT_REQ_CREDIT_BUREAU_%s' % i for i in ['HOUR', 'DAY', 'WEEK', 'MON', 'QRT', 'YEAR']]
REGION_GROUP = ['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_LIVE_REGION', 
                'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
                'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'REGION_POPULATION_RELATIVE']
APARTMENT_GROUP = ['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG',
                   'COMMONAREA_AVG', 'ELEVATORS_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG',
                   'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG',
                   'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
                   'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE',
                   'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE',
                   'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI',
                   'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI',
                   'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI',
                   'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
                   'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 'FONDKAPREMONT_MODE',
                   'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
SOCIAL_CIRCLE_GROUP = ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
                       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
APPR_PROCESS_GROUP = ['HOUR_APPR_PROCESS_START', 'WEEKDAY_APPR_PROCESS_START']
EXT_SOURCE_GROUP = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
OTHER_GROUP = ['ENTRANCES_AVG', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE']

In [10]:
ALL_COLUMNS = set(application.columns)
COLUMNS_IN_GROUPS = FLAG_DOCUMENT_GROUP + TARGET_GROUP + GENERAL_BIO_GROUP + DAYS_BIO_GROUP +\
                    AMT_GROUP + REGION_GROUP + APARTMENT_GROUP + SOCIAL_CIRCLE_GROUP +\
                    APPR_PROCESS_GROUP + EXT_SOURCE_GROUP + OTHER_GROUP
print(len(ALL_COLUMNS) == len(COLUMNS_IN_GROUPS), len(ALL_COLUMNS), len(COLUMNS_IN_GROUPS))
print(set(COLUMNS_IN_GROUPS) == set(ALL_COLUMNS), set(ALL_COLUMNS) - set(COLUMNS_IN_GROUPS))

True 122 122
True set()


In [19]:
CATEGORICAL_FEATURES = [
    'CODE_GENDER',
    'NAME_CONTRACT_TYPE',
    'FLAG_OWN_CAR',
    'CNT_CHILDREN',
    'NAME_INCOME_TYPE',
    'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS',
    'NAME_HOUSING_TYPE',
]

NUMERICAL_FEATURES = [
    'AMT_INCOME_TOTAL',
    'AMT_CREDIT', 
    'AMT_ANNUITY',
    'AMT_GOODS_PRICE',
    'REGION_POPULATION_RELATIVE',
    'DAYS_BIRTH',
    'DAYS_EMPLOYED',
]

In [20]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, NUMERICAL_FEATURES),
        ('cat', categorical_transformer, CATEGORICAL_FEATURES)])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
    application[NUMERICAL_FEATURES + CATEGORICAL_FEATURES], application['TARGET'], test_size=0.3, random_state=1)

In [36]:
clf.fit(X_train, y_train)
print('test: ', roc_auc_score(clf.predict(X_test), y_test))
print('train: ', roc_auc_score(clf.predict(X_train), y_train))

/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


test:  0.7095347526340893
train:  0.6382611745794289


In [7]:
model = LogisticRegression()

In [ ]:
model.fit()